In [ ]:
%load_ext autoreload
%autoreload 2

# Fitting Bond ASW and Z-Spread Curves

Fitting a bond curve to asset swap spreads and z-scores, using the range of parametric functions provided

In [ ]:
import datetime as dt
import pandas as pd
import numpy as np

In [ ]:
from financepy.utils import *
from financepy.products.bonds import *
from financepy.products.rates.ibor_single_curve import IborSingleCurve
from financepy.products.rates.ibor_benchmarks_report import dataframe_to_benchmarks

### Load up some Bond and Curve Data

In [ ]:
bond_dataframe = pd.read_csv('./data/giltbondprices.txt', sep='\t')
bond_dataframe['mid'] = 0.5*(bond_dataframe['bid'] + bond_dataframe['ask'])
bond_dataframe['maturity'] = pd.to_datetime(bond_dataframe['maturity'])

In [ ]:
bond_dataframe.head()
len(bond_dataframe)

In [ ]:
dfbm = pd.read_csv('./data/GBP_OIS_20120919.csv', index_col=0)
dfbm['base_date'] = pd.to_datetime(dfbm['base_date'], errors='ignore', format='%d/%m/%Y')
dfbm['start_date'] = pd.to_datetime(dfbm['start_date'], errors='ignore')  # allow tenors
dfbm['maturity_date'] = pd.to_datetime(dfbm['maturity_date'], errors='ignore')  # allow tenors

In [ ]:
valuation_date = from_datetime(dfbm.loc[0, 'base_date'])
cal = CalendarTypes.UNITED_KINGDOM
bms = dataframe_to_benchmarks(dfbm, asof_date=valuation_date, calendar_type=cal)
depos = bms['IborDeposit']
fras = bms['IborFRA']
swaps = bms['IborSwap']

fras.sort(key=lambda fra: fra.maturity_dt)
ois_curve = IborSingleCurve(valuation_date, depos, fras, swaps, InterpTypes.LINEAR_ZERO_RATES)

In [ ]:
# SPECIFY UK BOND CONVENTIONS
frequencyType = FrequencyTypes.SEMI_ANNUAL
accrual_type = DayCountTypes.ACT_ACT_ICMA
settlement = valuation_date

In [ ]:
bonds = []
asws = []
z_spreads = []

# LOAD BONDS AND CREATE A VECTOR OF FINBOND AND THEIR CORRESPONDING asms/z-spreads

for index, bondData in bond_dataframe.iterrows():

    date_string = bondData['maturity']
    matDatetime = bondData['maturity']
    maturity_dt = from_datetime(matDatetime)
    coupon = bondData['coupon']/100.0
    clean_price = bondData['mid']
    issue_date = Date(maturity_dt.d, maturity_dt.m, 2000)
    bond = Bond(issue_date, maturity_dt, coupon, frequencyType, accrual_type)

    z_spread = bond.z_spread(settlement, clean_price, ois_curve)
    asset_swap_spread = bond.asset_swap_spread(ois_curve.value_dt, clean_price, ois_curve)

    bonds.append(bond)
    asws.append(asset_swap_spread)    
    z_spreads.append(z_spread)

### Eyerball asset swap spreads and z-spreads

In [ ]:
plt.plot(bond_dataframe['maturity'], np.array(asws)*100, '.', label='asset_swap_spread')
plt.plot(bond_dataframe['maturity'], np.array(z_spreads)*100, '.', label='z_spread')
plt.legend(loc='best')
plt.show()


In [ ]:
plt.plot(np.array(asws)*100, np.array(z_spreads)*100, '.', label='z_spreads')
plt.plot(np.array(asws)*100, np.array(asws)*100, '--', label = 'x=y')
plt.xlabel('asset_swap_spread')
plt.ylabel('z_spread')
plt.legend(loc='best')
plt.show()


## Fitting the Bond Curves

### Cubic Polynomial

In [ ]:
# FIT THE ASWS AND Z-SPREADS TO A CUBIC POLYNOMIAL
curveFitMethod = CurveFitPolynomial()
fittedCurve1a = BondYieldCurve(settlement, bonds, asws, curveFitMethod)
fittedCurve1a.plot("GBP ASW Curve", ylabel = 'asset swap spreads (%)')
fittedCurve1z = BondYieldCurve(settlement, bonds, z_spreads, curveFitMethod)
fittedCurve1z.plot("GBP z-spread Curve", ylabel = 'z-spreads (%)')

See the cubic coefficients

In [ ]:
print(fittedCurve1a.curve_fit)

### Quintic Polynomial

In [ ]:
# FIT THE BOND ASWs and Z-SPREADs TO A QUINTIC POLYNOMIAL
curveFitMethod = CurveFitPolynomial(5)
fittedCurve2a = BondYieldCurve(settlement, bonds, asws, curveFitMethod)
fittedCurve2a.plot("GBP ASW Curve", ylabel = 'asset swap spreads (%)')
fittedCurve2z = BondYieldCurve(settlement, bonds, z_spreads, curveFitMethod)
fittedCurve2z.plot("GBP z-spread Curve", ylabel = 'z-spreads (%)')

See the quintic coefficients

In [ ]:
print(fittedCurve2z.curve_fit)

### Nelson-Siegel Curve

In [ ]:
# FIT THE BONDS TO A NELSON-SIEGEL CURVE
curveFitMethod = CurveFitNelsonSiegel()
fittedCurve3a = BondYieldCurve(settlement, bonds, asws, curveFitMethod)
fittedCurve3a.plot("GBP ASW Curve", ylabel = 'asset swap spreads (%)')
fittedCurve3z = BondYieldCurve(settlement, bonds, z_spreads, curveFitMethod)
fittedCurve3z.plot("GBP z-spread Curve", ylabel = 'z-spreads (%)')

In [ ]:
print(fittedCurve3a.curve_fit)

### Nelson-Siegel Svensson Curve

In [ ]:
# FIT THE BONDS TO A NELSON-SIEGEL-SVENSSON CURVE
curveFitMethod = CurveFitNelsonSiegelSvensson()
fittedCurve4a = BondYieldCurve(settlement, bonds, asws, curveFitMethod)
fittedCurve4a.plot("GBP ASW Curve", ylabel = 'asset swap spreads (%)')
fittedCurve4z = BondYieldCurve(settlement, bonds, z_spreads, curveFitMethod)
fittedCurve4z.plot("GBP z-spread Curve", ylabel = 'z-spreads (%)')

In [ ]:
print(fittedCurve4a.curve_fit)

### B-Spline Curve

In [ ]:
# FIT THE BONDS TO A B-SPLINE CURVE
curveFitMethod = CurveFitBSpline(power = 3, knots = [2, 3,5,10,15,20,30,40])
fittedCurve5a = BondYieldCurve(settlement, bonds, asws, curveFitMethod)
fittedCurve5a.plot("GBP ASW Curve", ylabel = 'asset swap spreads (%)')
fittedCurve5z = BondYieldCurve(settlement, bonds, z_spreads, curveFitMethod)
fittedCurve5z.plot("GBP z-spread Curve", ylabel = 'z-spreads (%)')

In [ ]:
print(fittedCurve5a.curve_fit)

## Accessing the Curve

You can access a yield from a date as follows

In [ ]:
# EXTRACT A YIELD FROM A FITTED YIELD CURVE
maturity_date = Date(19, 9, 2030)
interpolated_yield = fittedCurve5a.interp_yield(maturity_date)
print(maturity_date, interpolated_yield*100)

## Vectorisations

You can also access the curve using a time and this can be vectorised.

In [ ]:
times = np.linspace(0,10,100)

In [ ]:
interpolated_yields = fittedCurve5z.interp_yield(times)

In [ ]:
interpolated_yields*100